## <span style="color:red">Homework 2 </span>

# Disclaimer
* This notebook has been written by Benjamin Amar in 2023 DSAIS class at Em-Lyon.
* Generative AI (ChatGPT 4) has been used for debugging, inspiration.

**I- Reading Assignment**

[James, G., Witten, D., Hastie, T., Tibshirani, R., & Taylor, J. (2023). An Introduction to Statistical Learning : With Applications in Python (1st ed. 2023 edition). Springer.](https://www.statlearning.com/)

- Read Chapter 8, Section 1 (8.1: pages 331-342)


## 0 - Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as m
import seaborn as sns
import import_ipynb

from scipy.stats import chi2_contingency
from itertools import combinations
from scipy import stats

**II- Binary Classification**

1- Select your binary outcome

Amongst your categorical variables, select a binary variable to be used as an outcome.
It you have no binary variable, then create a binary variable from one of your categorical variables.

2- Divide your dataset into a test sample and a training sample with a ratio of 20% vs 80% respectively. <br>
Be careful to do so that your outcomes remain representative of the initial data set and well shuffled.

3- Select one classifier and fit it on the training set then estimate its score both on the training set and the test set. <br>Use the accuracy, the ROC AUC, the F1 score and the Kappa to estimate the performance of your classifier.

4- Repeat the same code on two other different classifiers: 

- Recommendation: create a function that does all the job and run it using three different classifiers
. 
- Which is the best classifier ?

In [2]:
path = "Toydataset.csv"
data = pd.DataFrame()

if path.endswith('.csv'):
    data = pd.read_csv(path, index_col=0)

elif path.endswith('.xlsx') or path.endswith('.xls'):
    data = pd.read_excel(path)

elif path.endswith('.txt'):
    data = pd.read_csv(path, sep='*', encoding='latin')

else:
    print(f"Unsupported file format. Please provide a .csv or .xlsx file")
    
if data is not None:
    print(f"Dataframe {path} loaded successfully! 👍")

Dataframe Toydataset.csv loaded successfully! 👍


In [3]:
datac = data.copy()

In [4]:
datac.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 18600 entries, 1.0 to 200.0
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Game              16537 non-null  object 
 1   Month_str         16549 non-null  object 
 2   Month             16538 non-null  float64
 3   Year              16510 non-null  float64
 4   YearTop1          18600 non-null  bool   
 5   Hours_watched     16468 non-null  float64
 6   Hours_streamed    16451 non-null  float64
 7   Peak_viewers      16425 non-null  float64
 8   Peak_channels     16494 non-null  float64
 9   Streamers         16453 non-null  float64
 10  Avg_viewers       16453 non-null  float64
 11  Avg_channels      16552 non-null  float64
 12  Avg_viewer_ratio  16515 non-null  float64
dtypes: bool(1), float64(10), object(2)
memory usage: 1.9+ MB


In [5]:
datac

,Game,Month_str,Month,Year,YearTop1,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
Rank,,,,,,,,,,,,,
1.0,League of Legends,NaN,1.0,2016.0,False,94377226.0,1362044.0,530270.0,2903.0,129172.0,127021.0,NaN,69.29
2.0,Counter-Strike: Global Offensive,January,1.0,2016.0,True,47832863.0,830105.0,372654.0,2197.0,NaN,64378.0,1117.0,57.62
3.0,Dota 2,January,1.0,2016.0,False,45185893.0,433397.0,NaN,1100.0,44074.0,60815.0,583.0,104.26
4.0,Hearthstone,January,1.0,2016.0,False,39936159.0,235903.0,NaN,NaN,36170.0,53749.0,317.0,169.29
5.0,Call of Duty: Black Ops III,January,1.0,2016.0,False,16153057.0,1151578.0,71639.0,NaN,214054.0,21740.0,1549.0,14.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196.0,PlateUp!,NaN,9.0,2023.0,False,560064.0,18617.0,16476.0,71.0,4034.0,778.0,25.0,30.08
197.0,PokÃ©mon GO,September,9.0,2023.0,False,551596.0,16578.0,3001.0,NaN,NaN,767.0,23.0,33.27
NaN,Bloons TD 6,September,9.0,2023.0,False,NaN,20142.0,10320.0,65.0,5673.0,752.0,28.0,26.85


In [6]:
# Initialize a dictionary to store type consistency results
type_consistency = {}

# Iterate through DataFrame columns
for column in datac:
    # Get the unique data types in the column
    unique_types = datac[column].apply(type).unique()
    
    # Check if there's more than one unique data type
    if len(unique_types) == 1:
        type_consistency[column] = f"🟢 {column}: Consistent ({unique_types[0].__name__})"
    else:
        type_consistency[column] = f"🔴 {column}: Inconsistent ({', '.join(t.__name__ for t in unique_types)})"

# Print the type consistency results for each feature
for consistency in type_consistency.values():
    print(consistency)
    #output_file.write(consistency)

🔴 Game: Inconsistent (str, float)
🔴 Month_str: Inconsistent (float, str)
🟢 Month: Consistent (float)
🟢 Year: Consistent (float)
🟢 YearTop1: Consistent (bool)
🟢 Hours_watched: Consistent (float)
🟢 Hours_streamed: Consistent (float)
🟢 Peak_viewers: Consistent (float)
🟢 Peak_channels: Consistent (float)
🟢 Streamers: Consistent (float)
🟢 Avg_viewers: Consistent (float)
🟢 Avg_channels: Consistent (float)
🟢 Avg_viewer_ratio: Consistent (float)


In [7]:
datac.isna().sum().sum()

25255

Type inconsistency is tied to the NaN values we have in our dataset

We need to manage that with the KNN method.

# 1 - Encoding

## 1.1 - KNN Imputing for quantitative data

In [8]:
from sklearn.impute import KNNImputer

# Selecting only the quantitative columns
quantitative_cols = datac.select_dtypes(include=['float64', 'int64']).columns

imputer = KNNImputer(n_neighbors=5)

datac[quantitative_cols] = imputer.fit_transform(datac[quantitative_cols])

In [9]:
datac[quantitative_cols].isna().sum()

Month               0
Year                0
Hours_watched       0
Hours_streamed      0
Peak_viewers        0
Peak_channels       0
Streamers           0
Avg_viewers         0
Avg_channels        0
Avg_viewer_ratio    0
dtype: int64

## 1.2 - KNN Imputing for qualitative data

In [10]:
"""
# Step 1: One-Hot Encode
datac_encoded = pd.get_dummies(datac, columns=datac.select_dtypes(include=['object']).columns, drop_first=False)

# Step 2: Apply KNN Imputation
imputer = KNNImputer(n_neighbors=5)
datac_imputed = pd.DataFrame(imputer.fit_transform(datac_encoded), columns=datac_encoded.columns)

# Step 3: Post-process the Imputed Values
for col in datac.select_dtypes(include=['object']).columns:
    dummies = [col_ for col_ in datac_encoded.columns if col_.startswith(col + "_")]
    # For each row, find the column with the highest imputed value
    max_col = datac_imputed[dummies].idxmax(axis=1)
    # Strip the prefix to get original category
    datac[col] = max_col.str.replace(col + "_", "")

# Step 4: Drop the one-hot encoded columns from the original dataframe as they're no longer needed
datac.drop(columns=datac_encoded.columns.difference(datac.columns), inplace=True)
"""


'\n# Step 1: One-Hot Encode\ndatac_encoded = pd.get_dummies(datac, columns=datac.select_dtypes(include=[\'object\']).columns, drop_first=False)\n\n# Step 2: Apply KNN Imputation\nimputer = KNNImputer(n_neighbors=5)\ndatac_imputed = pd.DataFrame(imputer.fit_transform(datac_encoded), columns=datac_encoded.columns)\n\n# Step 3: Post-process the Imputed Values\nfor col in datac.select_dtypes(include=[\'object\']).columns:\n    dummies = [col_ for col_ in datac_encoded.columns if col_.startswith(col + "_")]\n    # For each row, find the column with the highest imputed value\n    max_col = datac_imputed[dummies].idxmax(axis=1)\n    # Strip the prefix to get original category\n    datac[col] = max_col.str.replace(col + "_", "")\n\n# Step 4: Drop the one-hot encoded columns from the original dataframe as they\'re no longer needed\ndatac.drop(columns=datac_encoded.columns.difference(datac.columns), inplace=True)\n'

KNN Imputing for qualitative data was too complicated to implement without issues.
I could try using label encoding before using the KNN Imputing however my categorical data is nominal and not ordinal.

Therefore I have decided to use a different method :
- **Mode imputation** : No. I don't want to bias my model with the most frequent value since there is no reason to replace every NaN (11% of the dataset) with this value.
- **Random imputation** : No. I am creating false information in my dataset that can confuse my model.
- **Drop rows** : Possible. This is a possibility I was thinking about as a last resort if I couldn't find an other solution, since it's very destructive.
- **Constant imputation** : Yes. The method I am going with. I am going to accept the fact that I don't know certain 'Game' data and replace these NaNs with 'Unknown', thus my model will still train with 100% of the data I give it. The model will maybe predict 'Unkown' data but I prefer predicting that I can't be certain of an information than predicting a false information.

## 1.3 - Constant Imputation for qualitative data

In [11]:
# Selecting only the categorical columns
categorical_columns = datac.select_dtypes(include=['object']).columns

# Replace NaN values in categorical columns with 'Unknown'
datac[categorical_columns] = datac[categorical_columns].fillna('Unknown')

In [12]:
datac

,Game,Month_str,Month,Year,YearTop1,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
Rank,,,,,,,,,,,,,
1.0,League of Legends,Unknown,1.0,2016.0,False,94377226.0,1362044.0,530270.0,2903.0,129172.0,127021.0,1742.8,69.290
2.0,Counter-Strike: Global Offensive,January,1.0,2016.0,True,47832863.0,830105.0,372654.0,2197.0,30256.6,64378.0,1117.0,57.620
3.0,Dota 2,January,1.0,2016.0,False,45185893.0,433397.0,169907.0,1100.0,44074.0,60815.0,583.0,104.260
4.0,Hearthstone,January,1.0,2016.0,False,39936159.0,235903.0,94691.4,619.4,36170.0,53749.0,317.0,169.290
5.0,Call of Duty: Black Ops III,January,1.0,2016.0,False,16153057.0,1151578.0,71639.0,484.2,214054.0,21740.0,1549.0,14.030
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196.0,PlateUp!,Unknown,9.0,2023.0,False,560064.0,18617.0,16476.0,71.0,4034.0,778.0,25.0,30.080
197.0,PokÃ©mon GO,September,9.0,2023.0,False,551596.0,16578.0,3001.0,121.6,4070.8,767.0,23.0,33.270
NaN,Bloons TD 6,September,9.0,2023.0,False,444984.0,20142.0,10320.0,65.0,5673.0,752.0,28.0,26.850


In [13]:
datac.isna().sum().sum()

0

Nice! We don't have any missing values anymore, we can now start modelling!

In [14]:
datac_encode = datac.copy()

In [15]:
datac_encode.drop('Month_str', axis=1, inplace=True)

In [16]:
datac_encode = pd.get_dummies(datac_encode, columns=['Game'], drop_first=True)

In [17]:
datac_encode.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 18600 entries, 1.0 to 200.0
Columns: 2055 entries, Month to Game_theHunter: Call of the Wild
dtypes: bool(1), float64(10), uint8(2044)
memory usage: 37.8 MB


# 2 - Modeling

Our binary outcome is going to be 'YearTop1' which is assessing if the selected 'Game' had the highest average concurrent viewership of the said year.

## 2.1 - Dataset Division

In [18]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets with 80% and 20% of the data respectively
train_datac, test_datac = train_test_split(datac_encode, test_size=0.2, random_state=42)

print("Training data shape:", train_datac.shape)
print("Test data shape:", test_datac.shape)

Training data shape: (14880, 2055)
Test data shape: (3720, 2055)


## 2.2 - Training

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, cohen_kappa_score

# Assuming the target variable is named 'target'. Please replace 'target' with the correct column name if different.
y_train = train_datac['YearTop1']
X_train = train_datac.drop('YearTop1', axis=1)
y_test = test_datac['YearTop1']
X_test = test_datac.drop('YearTop1', axis=1)

# Train the Random Forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on training and test sets
train_preds = clf.predict(X_train)
test_preds = clf.predict(X_test)

# Calculate performance metrics for training data
train_accuracy = accuracy_score(y_train, train_preds)
train_roc_auc = roc_auc_score(y_train, train_preds)
train_f1 = f1_score(y_train, train_preds)
train_kappa = cohen_kappa_score(y_train, train_preds)

# Calculate performance metrics for test data
test_accuracy = accuracy_score(y_test, test_preds)
test_roc_auc = roc_auc_score(y_test, test_preds)
test_f1 = f1_score(y_test, test_preds)
test_kappa = cohen_kappa_score(y_test, test_preds)

print(f"Training Data Metrics:\nAccuracy: {train_accuracy}\nROC AUC: {train_roc_auc}\nF1 Score: {train_f1}\nCohen's Kappa: {train_kappa}")
print("\n")
print(f"Test Data Metrics:\nAccuracy: {test_accuracy}\nROC AUC: {test_roc_auc}\nF1 Score: {test_f1}\nCohen's Kappa: {test_kappa}")

Training Data Metrics:
Accuracy: 1.0
ROC AUC: 1.0
F1 Score: 1.0
Cohen's Kappa: 1.0


Test Data Metrics:
Accuracy: 0.8798387096774194
ROC AUC: 0.4989329268292683
F1 Score: 0.0
Cohen's Kappa: -0.003718279933360602


Let's build a function that automates the training of different models.

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

def evaluate_classifier(classifier, X_train, y_train, X_test, y_test):
    """
    Function : Train the classifier and compute metrics.
    """
    
    # Fit the classifier
    classifier.fit(X_train, y_train)
    
    # Predict on training and test sets
    train_preds = classifier.predict(X_train)
    test_preds = classifier.predict(X_test)

    # Calculate performance metrics for training data
    train_accuracy = accuracy_score(y_train, train_preds)
    train_roc_auc = roc_auc_score(y_train, train_preds)
    train_f1 = f1_score(y_train, train_preds)
    train_kappa = cohen_kappa_score(y_train, train_preds)

    # Calculate performance metrics for test data
    test_accuracy = accuracy_score(y_test, test_preds)
    test_roc_auc = roc_auc_score(y_test, test_preds)
    test_f1 = f1_score(y_test, test_preds)
    test_kappa = cohen_kappa_score(y_test, test_preds)

    # Print results
    print(f"{classifier.__class__.__name__} Metrics:")
    print(f"Training Data: Accuracy={train_accuracy}, ROC AUC={train_roc_auc}, F1 Score={train_f1}, Cohen's Kappa={train_kappa}")
    print(f"Test Data: Accuracy={test_accuracy}, ROC AUC={test_roc_auc}, F1 Score={test_f1}, Cohen's Kappa={test_kappa}")
    print("\n")
    
    return test_accuracy, test_roc_auc, test_f1, test_kappa


# Initialize classifiers
rf = RandomForestClassifier(random_state=42)
lr = LogisticRegression(random_state=42, max_iter=10000)
gb = GradientBoostingClassifier(random_state=42)

# Evaluate classifiers
results = {}
results["RandomForest"] = evaluate_classifier(rf, X_train, y_train, X_test, y_test)
results["LogisticRegression"] = evaluate_classifier(lr, X_train, y_train, X_test, y_test)
results["GradientBoosting"] = evaluate_classifier(gb, X_train, y_train, X_test, y_test)

# Determine the best classifier based on test ROC AUC
best_classifier = max(results, key=lambda k: results[k][1])
print(f"The best classifier is {best_classifier} based on ROC AUC score.")


RandomForestClassifier Metrics:
Training Data: Accuracy=1.0, ROC AUC=1.0
Test Data: Accuracy=0.8798387096774194, ROC AUC=0.4989329268292683


LogisticRegression Metrics:
Training Data: Accuracy=0.8846102150537635, ROC AUC=0.5
Test Data: Accuracy=0.8817204301075269, ROC AUC=0.5


GradientBoostingClassifier Metrics:
Training Data: Accuracy=0.8854166666666666, ROC AUC=0.5034944670937682
Test Data: Accuracy=0.8811827956989248, ROC AUC=0.4996951219512195


The best classifier is LogisticRegression based on ROC AUC score.


**III- Multiclass Classification** (Optional with good bonus point)

This time, select a categorical variable having more than 2 categories. I you have none, then categorize one of your quantitative data (e.g in using `pd.cut()` and the corresponding quantiles) into at least three classes.

Copy the codes of PART I and adjust to the new outcome. Use only Accuracy and F1 Score for performance (or sklearn `classification_report`)

# 3 - Multiclass Classification

For multiclass classification of a categorical variable, we are going to need to stratify our data before splitting so we keep the same data distribution between our values in our training dataset and testing dataset.

## 3.1 - Dataset Division

In [21]:
# Filter out categories in 'Game' with less than 3 instances (needed for stratification!)
datac_filtered = datac[datac.groupby('Game')['Game'].transform('size') > 2]

# Perform the stratified split on the filtered data
y_filtered = datac_filtered['Game']
X_filtered = datac_filtered.drop('Game', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42, stratify=y_filtered)

print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

Training data shape: (13644, 12)
Test data shape: (3411, 12)


In [22]:
# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Perform dummy encoding for the training data
X_train_encoded = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
y_train_encoded = pd.get_dummies(y_train, columns=categorical_cols, drop_first=True)

# Perform dummy encoding for the test data
# Note: Use the columns from the training data to ensure consistency
X_test_encoded = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True).reindex(columns=X_train_encoded.columns, fill_value=0)
y_test_encoded = pd.get_dummies(y_test, columns=categorical_cols, drop_first=True).reindex(columns=y_train_encoded.columns, fill_value=0)

print("Encoded Training data shape:", X_train_encoded.shape)
print("Encoded Test data shape:", X_test_encoded.shape)

Encoded Training data shape: (13644, 23)
Encoded Test data shape: (3411, 23)


## 2.2 - Training

In [23]:
# Evaluate classifiers using the previously defined function
results_filtered = {}
"""
results_filtered["RandomForest"] = evaluate_classifier(rf, X_train_encoded, y_train_encoded, X_test_encoded, y_test_encoded)
results_filtered["LogisticRegression"] = evaluate_classifier(lr, X_train_encoded, y_train_encoded, X_test_encoded, y_test_encoded)
results_filtered["GradientBoosting"] = evaluate_classifier(gb, X_train_encoded, y_train_encoded, X_test_encoded, y_test_encoded)
"""

# Determine the best classifier based on test Accuracy
best_classifier_filtered = max(results_filtered, key=lambda k: results_filtered[k][0])
print(f"The best classifier is {best_classifier_filtered} based on Accuracy score.")

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.